# **Experimento prompts**
Generar Datos de respuesta de los PROMPTS y almacenarlos SQLITE para su posterior analisis

In [58]:
#imports
import pandas as pd
import sqlite3
from ollama import Client
import datetime

In [60]:
#Funciones
def get_list_tbl(tbl_name):
    """
    The function `get_list_tbl` retrieves the names of tables in a SQLite database that match a
    specified name.
    
    :param tbl_name: The `tbl_name` parameter in the `get_list_tbl` function is used to specify the name
    of the table for which you want to retrieve information from the SQLite database. The function will
    query the database to fetch the names of tables that match the provided `tbl_name`
    :return: The function `get_list_tbl(tbl_name)` returns a DataFrame containing the names of tables in
    a SQLite database that match the input table name `tbl_name`.
    """
    df_tbls_names = pd.read_sql_query(f"SELECT name FROM sqlite_master where name='{tbl_name}'")
    return df_tbls_names

# get_response_from_llm('https://9f0c-34-126-167-50.ngrok-free.app', 'llama2/llama3/gemma:7b', text_prompt)
def get_response_from_llm(client, llm_model, text_prompt): #url:
    """
    The function `get_response_from_llm` takes in a URL host, a GPT-3 model, and a text prompt, then
    uses a client to chat with the specified model and returns the response content.
    
    :param url_host: The `url_host` parameter refers to the host URL where the language model API is
    hosted. This is the endpoint that the client will connect to in order to interact with the language
    model
    :param llm_model: The `llm_model` parameter refers to the specific language model that will be used
    for generating responses. It could be a model like `llama2`, `llama3`, or `gemma:7b`. This parameter
    determines the characteristics and capabilities of the language model being utilized for the
    :param text_prompt: The `text_prompt` parameter is the text input or prompt that you want to send to
    the language model for generating a response. It is the message that you want the model to respond
    to
    :return: The function `get_response_from_llm` is returning the content of the response message from
    the language model (LLM) after sending a text prompt to it.
    """

    #client = Client(host=url_host) # este hay que cambiarlo por el que da Colab
    response = client.chat(model=llm_model # llama2/llama3/gemma:7b
                           , messages=[
      {
        'role': 'user',
        'content': text_prompt,
      },
    ])

    return response['message']['content']


def get_prompts_from_tbl(df,conn):
    """
    The function `get_prompts_from_tbl` iterates through a DataFrame, prints a header for each row, and
    then retrieves the first 3 rows from a table in a database connection.
    
    :param df: The `df` parameter is a DataFrame containing information about tables in a database. It
    is used to iterate over each row to retrieve prompts from each table
    :param conn: The `conn` parameter in the function `get_prompts_from_tbl` is likely a database
    connection object that allows you to interact with a database. It is used to execute SQL queries
    against the database specified in the connection
    :return: The function `get_prompts_from_tbl` is returning the DataFrame `df_prompts` which contains
    the result of the SQL query selecting all columns from the table specified in the `name` column of
    the input DataFrame `df`, limited to 3 rows.
    """

    for _, row in df.iterrows(): # no uso info de index
      print(f"\n#### {row['name']} ####\n")
      df_prompts = pd.read_sql_query(f"SELECT * FROM {row['name']} limit 11", conn)
    return df_prompts


def get_prompts_answer(df_prompts, ollama_client, model_llm):
    """
    The function `get_prompts_answer` iterates through prompts in a DataFrame, generates responses using
    a language model, and returns a DataFrame with prompt IDs and corresponding answers.
    
    :param df_prompts: The `df_prompts` parameter is likely a DataFrame containing prompts with their
    corresponding IDs. The function `get_prompts_answer` iterates over each row in this DataFrame,
    retrieves a response using the OpenAI Language Model (OLLAMA), and stores the prompt ID and response
    in a dictionary
    :param ollama_client: The `ollama_client` parameter is likely an object or client used to interact
    with the OpenAI Language Model API. It is used in the `get_response_from_llm` function to make
    requests to the language model and retrieve responses based on the provided prompts
    :return: The function `get_prompts_answer` returns a pandas DataFrame containing two columns:
    'id_prompts' and 'answers'. The 'id_prompts' column contains a list of prompt IDs from the input
    DataFrame `df_prompts`, and the 'answers' column contains a list of responses generated by the
    `get_response_from_llm` function using the prompts provided in the 'prompt' column of
    """

    dict_prompt_answer = {}
    list_id_prompts = []
    list_prompt_answers = []      
      
    for index, row in df_prompts.iterrows():
      print(row['id_promtp'], row['prompt'])
      #####
      llm_reponse = get_response_from_llm(client = ollama_client
                                          , llm_model = model_llm #'llama2' # 'llama2/llama3/gemma:7b'
                                          , text_prompt = row['prompt']
                                          )
      #####
       
      list_id_prompts.append(row['id_promtp'])
      list_prompt_answers.append(llm_reponse)
    
    dict_prompt_answer['id_prompts'] = list_id_prompts
    dict_prompt_answer['answers'] = list_prompt_answers
    return pd.DataFrame(dict_prompt_answer)


### **Main**

In [ ]:
if __name__ == '__main__':

    """
    Main function that runs the script.
    It connects to a SQLite database, retrieves a table and a model name,
    creates a client for the OpenAI Language Model API, retrieves prompts from the
    table and their corresponding answers, and stores the results in a new table in the database.
    """

    begin_time = datetime.datetime.now()

    # Defino la ubicacion de la bbdd Sqlite a utilizar
    con = sqlite3.connect('/mnt/e/Facultad/Licenciatura en Ciencia de Datos-UGR/Cursada/Tercer Cuatrimestre/'
                      'Seminario_Trabajo_Final/Test_IA_models/Prompts/db/prompt_sqlite.db')
    # Defino el nombre de la tabla que se utilizara
    tbl = 'set_Extra_Muy_Alto'
    # Defino el nombre del modelo a utilizar 'llama2/gemma:7b'
    model_llm = 'gemma:7b' 

    # Obtengo los nombres de las tablas
    df_tbls = get_list_tbl(tbl)
    # Obtengo los prompts a enviar al modelo
    df_prompts = get_prompts_from_tbl(df_tbls, con)
    
    # Configuro el Cliente con la URL del servidor donde se encuentra el modelo
    url_host = 'https://86f6-34-16-199-170.ngrok-free.app'
    ollama_client = Client(host=url_host)
    
    # Obtengo un dataframe con las respuestas que otorga el modelo 
    df_prompts_answer = get_prompts_answer(df_prompts, ollama_client, model_llm)

    # Creo la tabla donde se guardaran los resultados(respuestas) a los prompts
    print(f'\nCreando tabla {tbl}_{model_llm}')
    df_prompts_answer.to_sql(f'{tbl}_{model_llm}', con, if_exists='replace', index=False)
    print(f'Tabla {tbl}_{model_llm} creada correctamente')

    end_time = datetime.datetime.now()
    print('tiempo de ejecucion:' + str(end_time-begin_time))

####  Verificacion ####

Verifico el Dataframe con las respuestas dadas por el modelo, los cuales se almacenaron en la tabla 

In [63]:
df_prompts_answer

,id_prompts,answers
0,1,\n¡Buenos días! ¡Claro que sí! Estoy aquí para...
1,2,\nLa capital de Francia es París.
2,3,\n¡Por supuesto! Argentina es un país fascinan...
3,4,\nInteligencia Artificial (IA) es el campo de ...
4,5,\nSure! Here is the translation of the text in...
5,6,"\nEl aire es libre, la brisa del mar\nUna sens..."
6,7,"Título: ""The Audition""\n\nAct 1:\n\nThe scene ..."
7,8,"```\nprint(""Hola, mundo!"")\n```\nEste código u..."
8,9,\n ¡Claro! Para analizar este conjunto de dato...
9,10,\nPuedes crear una función en PySpark utilizan...
